<a href="https://colab.research.google.com/github/SkyHaines/IMLO/blob/main/Copy_of_Coursework%3F%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The CIFAR 10 dataset contains 60,000 3x32x32 colour images, with 10 classes.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

# Need to look into how this works, perhaps i could tweak it to improve accuracy?
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Download training and testing data
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transform
)

100%|██████████| 170M/170M [00:02<00:00, 76.1MB/s]


Now, I need to experiment with batch size. (The number of images processed before updating the weights).

As I understand it,
Larger batch size : Faster training, lower accuracy
Smaller batch size: Slower training, higher accuracy

So I'm going to do some trial and error I think, to get the batch size at a point where I use up just under the whole 4 training hours, with the highest accuracy :)

Contextualise: There are 50,000 training imgs, so I'm going to try a batch size of 100 for now? (As its a multiple, I assume it helps somehow?)
UPDATED: Changed to 64, research suggests it helps with runtime to have it as a power of 2, for parallelism. Will take this into account for further optimisation.

In [2]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

classes = ('airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck')

Now I need to define the nn architecture/structure? I think I need to use a CNN rather than a regular NN because it's for images rather than numerical data

In [3]:
device = "cpu"
class MyCNN(nn.Module):
  def __init__(self):
    super().__init__()

    # First layer? Has 3 input channels due to RGB
    # input tensor = size of prev layer output * filters in prev layer
    # output tensor = size of input img * filters
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=24, kernel_size=8, stride=1, padding=1)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
    self.conv2 = nn.Conv2d(in_channels=24, out_channels=48,kernel_size=4, stride=1, padding=1)
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
    self.conv3 = nn.Conv2d(in_channels=48, out_channels=96, kernel_size=3, stride=1, padding=1)
    self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    self.fc1 = nn.Linear(in_features=96*3*3, out_features=512)
    self.fc2 = nn.Linear(in_features=512, out_features=256)
    self.fc3 = nn.Linear(in_features=256, out_features=128)
    self.fc4 = nn.Linear(in_features=128, out_features=10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.pool(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool2(x)
    x = self.conv3(x)
    x = F.relu(x)
    x = self.pool3(x)
    x = torch.flatten(x,1)
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    x = self.fc4(x)
    return x

model = MyCNN().to(device)
print(model)

MyCNN(
  (conv1): Conv2d(3, 24, kernel_size=(8, 8), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(24, 48, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=864, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=10, bias=True)
)


In [4]:
loss = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
def train(dataloader, model, loss_fn, optimiser):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      optimiser.zero_grad()

      pred = model(X)
      loss = loss_fn(pred, y)
      loss.backward()
      optimiser.step()

      if batch % 100 == 0:
        loss, current = loss.item(), (batch + 1) * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
epochs = 25
for t in range(epochs):
  print(f"Epoch {t+1} \n-------------------------------")
  train(train_dataloader, model, loss, optimiser)
  test(test_dataloader, model, loss)
print("Done :)")

Epoch 1 
-------------------------------
loss: 2.301523  [   64/50000]
loss: 1.792015  [ 6464/50000]
loss: 1.589204  [12864/50000]
loss: 1.590184  [19264/50000]
loss: 1.370969  [25664/50000]
loss: 1.373400  [32064/50000]
loss: 1.482574  [38464/50000]
loss: 1.192235  [44864/50000]
Test error: 
 Accuracy: 53.3%, Avg loss: 1.278532 

Epoch 2 
-------------------------------
loss: 1.075395  [   64/50000]
loss: 1.256635  [ 6464/50000]
loss: 1.421484  [12864/50000]
loss: 1.139790  [19264/50000]
loss: 1.047006  [25664/50000]
loss: 1.277868  [32064/50000]
loss: 1.020124  [38464/50000]
loss: 1.153665  [44864/50000]
Test error: 
 Accuracy: 60.0%, Avg loss: 1.131882 

Epoch 3 
-------------------------------
loss: 0.894919  [   64/50000]
loss: 1.009903  [ 6464/50000]
loss: 0.829539  [12864/50000]
loss: 1.114172  [19264/50000]
loss: 0.889152  [25664/50000]
loss: 0.758597  [32064/50000]
loss: 1.017440  [38464/50000]
loss: 1.028885  [44864/50000]
Test error: 
 Accuracy: 64.9%, Avg loss: 1.013412 

E

KeyboardInterrupt: 